In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d
from scipy.interpolate import griddata

### Plotting
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
### WRF data

### Parameters
nx = 400
ny = 250
nz = 44
ti = 16
lev = 400
levs = np.arange(300,510,10)

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
adj = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')
anl = Dataset('/p/work1/lloveras/nov2018/30km_files/gfs/wrfin_d01_2018-11-15_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat',timeidx=ti))
lons = np.asarray(getvar(ctl,'lon',timeidx=ti))
lons[lons >= 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure',timeidx=ti))
p_adj = np.asarray(getvar(adj,'pressure',timeidx=ti))
p_anl = np.asarray(getvar(anl,'pressure'))

### Height
z_ctl = np.asarray(getvar(ctl,'z',timeidx=ti))
z_adj = np.asarray(getvar(adj,'z',timeidx=ti))
z_anl = np.asarray(getvar(anl,'z'))

### PV
pv_ctl = np.asarray(getvar(ctl,'pvo',timeidx=ti))
pv_adj = np.asarray(getvar(adj,'pvo',timeidx=ti))
pv_anl = np.asarray(getvar(anl,'pvo'))

### Interpolate
zl_ctl = interplevel(z_ctl, p_ctl, lev)
zl_adj = interplevel(z_adj, p_adj, lev)
zl_anl = interplevel(z_anl, p_anl, lev)

pvl_ctl = interplevel(pv_ctl, p_ctl, lev)
pvl_adj = interplevel(pv_adj, p_adj, lev)
pvl_anl = interplevel(pv_anl, p_anl, lev)

zlay_ctl = np.mean(interplevel(z_ctl, p_ctl, levs),axis=0)
zlay_adj = np.mean(interplevel(z_adj, p_adj, levs),axis=0)
zlay_anl = np.mean(interplevel(z_anl, p_anl, levs),axis=0)

pvlay_ctl = np.mean(interplevel(pv_ctl, p_ctl, levs),axis=0)
pvlay_adj = np.mean(interplevel(pv_adj, p_adj, levs),axis=0)
pvlay_anl = np.mean(interplevel(pv_anl, p_anl, levs),axis=0)


In [ ]:
### COAMPS data

### Parameters
coamps_dir = '/p/work1/lloveras/nov2018/coamps_files/'
nx_c = 301
ny_c = 201
nz_c = 45
rf = np.fromfile(coamps_dir + 'rfbox1_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
rf = np.roll(np.flip(np.reshape(rf, [nz_c, ny_c, nx_c]),axis=0),-4,axis=-1)

### Latitude and longitude
lats_c = np.fromfile(coamps_dir + 'latitu_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lons_c = np.fromfile(coamps_dir + 'longit_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lats_c = np.reshape(lats_c, [ny_c, nx_c])
lons_c = np.reshape(lons_c, [ny_c, nx_c])
lons_c[lons_c > 0] -= 361

### Pressure
pc_ctl = (np.fromfile(coamps_dir + 'ttlprs_sig_028485_000010_1a0301x0201_2018111312_00480000_fcstfld', dtype='>f4'))
pc_ctl = np.flip(np.reshape(pc_ctl, [nz_c, ny_c, nx_c]), axis=0)
pc_adj = (np.fromfile(coamps_dir + '9x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00480000_fcstfld', dtype='>f4'))
pc_adj = np.flip(np.reshape(pc_adj, [nz_c, ny_c, nx_c]), axis=0)

### Height
zc_ctl = (np.fromfile(coamps_dir + 'geohgt_pre_000400_000000_1a0301x0201_2018111312_00480000_fcstfld', dtype='>f4',count=ny_c*nx_c))
zc_ctl = np.reshape(zc_ctl, [ny_c, nx_c])
zc_adj = (np.fromfile(coamps_dir + '9x/geohgt_pre_000400_000000_1a0301x0201_2018111312_00480000_fcstfld', dtype='>f4',count=ny_c*nx_c))
zc_adj = np.reshape(zc_adj, [ny_c, nx_c])

### PV
pvc_ctl = (np.fromfile(coamps_dir + 'potvor_sig_028485_000010_1a0301x0201_2018111312_00480000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pvc_ctl = np.flip(np.reshape(pvc_ctl, [nz_c, ny_c, nx_c]), axis=0)
pvc_adj = (np.fromfile(coamps_dir + '9x/potvor_sig_028485_000010_1a0301x0201_2018111312_00480000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pvc_adj = np.flip(np.reshape(pvc_adj, [nz_c, ny_c, nx_c]), axis=0)

### Interpolate PV to pressure levels and compute mean
pvclay_ctl = np.asarray(np.mean(interplevel(pvc_ctl, pc_ctl, levs),axis=0))
pvclay_adj = np.asarray(np.mean(interplevel(pvc_adj, pc_adj, levs),axis=0))

### Interpolate PV onto WRF horizontal grid to compute differences
pvclay_ctl_interp = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), pvclay_ctl.ravel(), (lons, lats), method='linear'))
pvclay_adj_interp = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), pvclay_adj.ravel(), (lons, lats), method='linear'))


In [ ]:
rf_w = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          rf[25,:,:].ravel(), (lons, lats), method='linear'))
pvcw = np.asarray(pvlay_ctl)
pvfw = np.asarray(pvlay_adj)
pva = np.asarray(pvlay_anl)
pvcw[rf_w < 1] = np.NaN
pvfw[rf_w < 1] = np.NaN
pva[rf_w < 1] = np.NaN
pvcc = np.asarray(pvclay_ctl_interp)
pvfc = np.asarray(pvclay_adj_interp)
pvcc[rf_w < 1] = np.NaN
pvfc[rf_w < 1] = np.NaN

In [ ]:
np.nanmean((pvfw - pva)**2)

In [ ]:
### Plot subdomain
min_lat = 24
max_lat = 45
min_lon = -105
max_lon = -75

### Contour intervals
zlvls = np.arange(0,1500,10)
dzlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.75

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-101,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['topleft','topright','cbar'],['bottomleft','bottomright','cbar']],
                              constrained_layout=True, figsize=(7,5.5), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,0.05],'height_ratios':[1,1]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels = False

axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl_anl,1)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['topleft'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['topleft'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['topleft'].contourf(lons, lats, pvlay_ctl - pvlay_anl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',
             transform=crs.PlateCarree())

axd['topleft'].contour(lons_c, lats_c, rf[25], levels=[0,1],colors='g',transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('WRF Control')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False
gls.bottom_labels=False
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_anl,1)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['topright'].contour(lons, lats, smooth2d(zl_adj,1)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['topright'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['topright'].contourf(lons, lats, pvlay_adj - pvlay_anl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',
             transform=crs.PlateCarree())

axd['topright'].contour(lons_c, lats_c, rf[25], levels=[0,1],colors='g',transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('WRF Perturbed')
axd['topright'].text(0.0, 1.0, '(b)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)


##############
### BOTTOM LEFT
#############

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.rotate_labels = False

axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl_anl,1)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['bottomleft'].contour(lons_c, lats_c, smooth2d(zc_ctl,1)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['bottomleft'].contourf(lons, lats, pvclay_ctl_interp - pvlay_anl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',
             transform=crs.PlateCarree())

axd['bottomleft'].contour(lons_c, lats_c, rf[25], levels=[0,1],colors='g',transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('COAMPS Control')
axd['bottomleft'].text(0.0, 1.0, '(c)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
#############

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl_anl,1)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['bottomright'].contour(lons_c, lats_c, smooth2d(zc_adj,1)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['bottomright'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['bottomright'].contourf(lons, lats, pvclay_adj_interp - pvlay_anl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',
             transform=crs.PlateCarree())

axd['bottomright'].contour(lons_c, lats_c, rf[25], levels=[0,1],colors='g',transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('COAMPS Perturbed')
axd['bottomright'].text(0.0, 1.0, '(d)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### Set the colorbar and show
cbar = fig.colorbar(im1, orientation='vertical', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig06_wrf_coamps_fcst.pdf',bbox_inches='tight')
plt.show()
